# Demo: loading data from Amazon into iguazio DB


In [1]:
!pip install v3io_frames

done
  Stored in directory: /igz/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
Successfully built msgpack-python


In [6]:
import pandas as pd
import v3io_frames as v3f
client = v3f.Client('http://v3io-framesd:8080')
tablename = 'bank'

In [7]:
# run only once (load SQL magic)
%load_ext sql
%config SqlMagic.autocommit=False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Step 1: Load data from Amazon S3

In [8]:
# read S3 file into a data frame and show its data & metadata
df = pd.read_csv('https://s3.amazonaws.com/apache-zeppelin/tutorial/bank/bank.csv', sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


## Step 2: Write data frames into the database using a single command
data is streamed into the database via fast NoSQL APIs

In [10]:
out = client.write('kv', tablename, df)

## Step 3: Read from the Database with DB side SQL
offload data filtering, grouping, joins, etc to a scale-out high speed DB engine

In [11]:
%sql select * from bigdata.bank2 where balance > 10000

Done.


loan,education,previous,housing,poutcome,duration,marital,default,balance,month,contact,campaign,y,job,day,age,pdays
no,secondary,0,no,unknown,219,married,no,26452,jul,telephone,2,no,retired,15,75,-1
no,secondary,0,yes,unknown,249,married,no,19317,aug,cellular,1,yes,retired,4,68,-1


## Read the data as a structured stream
the frames API returns dataframe iterator (a stream) to accelerate performance
and allow cuncurent data movement and processing

In [12]:
df = pd.concat(client.read(backend='kv', table=tablename, filter="balance>20000"), sort=False)
df.head(8)

,poutcome,age,housing,marital,y,campaign,previous,loan,balance,pdays,duration,month,job,contact,default,education,day
__name,,,,,,,,,,,,,,,,,
75,unknown,75,no,married,no,2,0,no,26452,-1,219,jul,retired,telephone,no,secondary,15


In [15]:
import pandas as pd
import v3io_frames as v3f
client = v3f.Client('http://localhost:8080', 's3cr3t')

In [ ]:
df = pd.concat(client.read(backend='kv', table='bank', filter="balance>5000"), sort=False)
df.head(8)